### **Transformers from Scratch using PyTorch**

**Required Libraries**

In [2]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install lightning

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch #To create tensors 
import torch.nn as nn  #for Module, Linear, Embedding
import torch.nn.functional as F #for softmax fn

from torch.optim import Adam #to fit the NN to data with backpropagation
from torch.utils.data import TensorDataset, DataLoader #tools to create a large scale Transformer Network with lots of training data

import lightning as L #for automatic code optimization and scaling in the cloud

**Create training dataset for the transformer**

In [5]:
token_to_id = {'what':0, 'is':1, 'CNN':2, 'Great':3, '<EOS>':4}
# These are the overall tokens in the prompt, we map them to numbers because PyTorch word embedding fn nn.Embedding() only accepts numbers.

In [6]:
# create a reverse dict to map from id back to token
id_to_token = dict(map(reversed, token_to_id.items()))
# These two dicts token_to_id and id_to_token makes it easier to format input to transformer and interpret output from Transformer


**Process the prompt**

It should start with "What" -> outputs "is","is" -> outputs "CNN", "CNN" -> outputs "<EOS>", <"<EOS>" -> outputs "Great", "Great" -> outputs "<EOS>"

In [7]:
#Input tensor 
inputs = torch.tensor([[token_to_id["what"],
                      token_to_id["is"],
                      token_to_id["CNN"],
                      token_to_id["<EOS>"],
                      token_to_id["Great"]], #For 1st prompt "What is CNN"

                      [token_to_id["CNN"],
                      token_to_id["is"],
                      token_to_id["what"],
                      token_to_id["<EOS>"],
                      token_to_id["Great"]]]) #For 2nd prompt "CNN is What"
#Labels - output from the decoder 
labels = torch.tensor([[token_to_id["is"],
                       token_to_id["CNN"],
                       token_to_id["<EOS>"],
                       token_to_id["Great"],
                       token_to_id["<EOS>"]], #For 1st prompt "What is CNN"

                       [token_to_id["is"],
                       token_to_id["CNN"],
                       token_to_id["<EOS>"],
                       token_to_id["Great"],
                       token_to_id["<EOS>"]]]) #For 2nd prompt "CNN is What"

# Pass the dataset to DataLoader to create dataloader
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

**Position Embedding**

PE (pos, 2i) = sin(pos/10000^(2i/d model)

PE (pos, 2i+1) = cos(pos/10000^(2i/d model) +1 in 2i+1 just 
means cosine comes after sine.
where pos -> position of the token
i -> each embedding position

Rather than computing the formula values for each tokens we pre-compute the y-axis values and store them in a matrix it make it much easier.

In [8]:
#To precompute and add positional encoding values to tokens
class PositionEncoding (nn.Module): #inherits from nn.module
    def __init__(self, d_model=2, max_len=6): #d_model - number of word embeddings per token
        #max_len is the max no of tokens our transformer can process (input and output combined)
        super().__init__()
        pe = torch.zeros(max_len, d_model) #create a matrix of positional encoding values and fill it with zeros
        # pe = max_len x d_model
        
        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1) #arange() to create a seq of nos between 0 & max_len
        #these numbers are float, unsqueeze(1) turns seq of nos into column matrix.
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()
        #step=2 results same seq no that we get if we multiplied i by 2.
        
        # div_term is the divisor in the formula above which divides the pos.
        div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe) #to move pe to GPU if we use one.

    def forward(self, word_embeddings): #takes word embeddings and adds positional embeddings
        return word_embeddings + self.pe[:word_embeddings.size(0), :]

**Masked Attention**

We need to calculate : Query , Key and Value for all tokens.
If we have a Matrix with WE+PE for each token (in each row) and we have weights to calculate query as matrix, then we can perform simple matrix multiplication. We can do the same for Key and Value.

In [9]:
class Attention(nn.Module):

    def __init__(self, d_model=2): # here d_model is needed in order to know the dimension of the weight matrices.

        super().__init__() 

        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False) #to create the Weight Mx for Q, use Linear()
        # Linear() here not just creates the weights will also do the math in further steps
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        #to keep track of rows and columns, useful to train input either sequentially or in batches
        self.row_dim = 0 
        self.col_dim = 1

    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
        # here encoding_for_q/k/v allows encodings to come from different sources (like in the case of encoder-decoder transformers)
        q = self.W_q(encodings_for_q) #we pass the encodings for each token
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim)) #calculates the similarity between Q and K

        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5) #scale sims by sq root of no of values used in each key
        #this scaling is a standard practice since the original transformers (Not required - only helps when model is large)

        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9) #adding a mask if we are using one
        # mask is optional its like masking the tokens to just see the previous tokens , there are other types of masks as well.
        # masked_fill only fills the value in the cells of the matrix whose attention values should be ignored 

        attention_percents = F.softmax(scaled_sims, dim=self.col_dim) #pass the similarities to softmax

        attention_scores = torch.matmul(attention_percents, v) #scale the V matrix with softmax outputs

        return attention_scores

**Building a decoder-only transformer** (Word Embedding + Position Embedding + Attention)

In [10]:
class DecoderOnlyTransformer(L.LightningModule): #inherit from lightning class

    def __init__(self, num_tokens=4, d_model=2, max_len=6):

        super().__init__()

        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model) #word embedding
        self.pe = PositionEncoding(d_model=d_model, max_len=max_len) #position embedding
        self.self_attention = Attention(d_model=d_model) #masked self-attention
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

        self.loss = nn.CrossEntropyLoss() #to quantify how well the model performs, this will apply the softmax fn itself

    def forward(self, token_ids):

        word_embeddings = self.we(token_ids) #convert tokens to word embeddings
        position_encoded = self.pe(word_embeddings)
        
        #defining the mask that prevents the tokens to look at succeeding tokens when we calculate Attention
        mask = torch.tril(torch.ones((token_ids.size(dim=0),token_ids.size(dim=0))))
        #first it creates a matrix will all 1s with symmetrical with dimensions of token size
        #this is passed to tril (lower-triangle), leaves 1s in lower triangle as it self and turns everything else to 0 (upper triangle values)
        mask = mask == 0 #to convert 0->True , 1->False in the matrix
        
        #calculating the attention
        self_attention_values = self.self_attention(position_encoded, position_encoded, position_encoded, mask=mask)
        #position_encoded passed thrice for Q, K and V.

        #Adding residual connections
        residual_connection_values = position_encoded + self_attention_values

        #pass to the FC layer
        fc_layer_output = self.fc_layer(residual_connection_values) #cross-entrophy does the softmax so we do not do it here again

        return fc_layer_output

    #code to train the transformer

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1) #passing all weights and biases to Adam, lr 0.1 for this specific model makes it very fast.

    def training_step(self, batch, batch_idx): #takes a batch of train data
        input_tokens, labels = batch #split data as inputs and labels
        output = self.forward(input_tokens[0]) #pass the inputs to the above forward method
        loss = self.loss(output, labels[0]) #compare output from transformer to true values

        return loss

In [16]:
#run training

model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

model_input = torch.tensor([token_to_id["what"], #create the prompt
                            token_to_id["is"],
                            token_to_id["CNN"],
                            token_to_id["<EOS>"]])

input_length = model_input.size(dim=0) #calculate token length

predictions = model(model_input) #makes prediction for what token comes next
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])]) #-1 -> index the output generated by the <EOS> token
#argmax is used to select the largest value form the prediction array
predicted_ids = predicted_id

max_length = 6
for i in range(input_length, max_length): #loop to keep generating the output until max_len or reaches <EOS>
    if (predicted_id == token_to_id["<EOS>"]):
        break
    
    model_input = torch.cat((model_input, predicted_id)) #Each time we generate a new o/p we add it to i/p so that each predicted is made with full context.
    # cat is for concatenation

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

    print("Predicted Tokens:\n")
    for id in predicted_ids:
        print("\t", id_to_token[id.item()]) #print generated tokens from converting id numbers to text.
    

Predicted Tokens:

	 Great
	 Great
Predicted Tokens:

	 Great
	 Great
	 <EOS>


In [15]:
    #training the model
    trainer = L.Trainer(max_epochs=30)
    trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ we             │ Embedding        │     10 │ train │     0 │
│ 1 │ pe             │ PositionEncoding │      0 │ train │     0 │
│ 2 │ self_attention │ Attention        │     12 │ train │     0 │
│ 3 │ fc_layer       │ Linear           │     15 │ train │     0 │
│ 4 │ loss           │ CrossEntropyLoss │      0 │ train │     0 │
└───┴────────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 37                                                                                               
Non-trainable params: 0                                                                                            
Total params: 37                                                                                                   
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 8                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

C:\Users\akshi\AppData\Roaming\Python\Python313\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
C:\Users\akshi\AppData\Roaming\Python\Python313\site-packages\lightning\pytorch\loops\fit_loop.py:317: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


`Trainer.fit` stopped: `max_epochs=30` reached.


Rerun the model (previous cell ) after training to get better predictions